In [1]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity
jieba.load_userdict(os.path.join('data', 'dict.txt.big'))

Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.394 seconds.
Prefix dict has been built succesfully.


In [2]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

## sort the contents by index
contents = contents[news_urls.News_URL]
keys, content_list = contents.keys(), contents.values

In [7]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        return jieba.lcut(x, cut_all=False)
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentenece_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [12]:
from sklearn.model_selection import train_test_split
batch_size = 100000
folder = 'news_data_1/'
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

## okapi model

In [20]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentenece_arr)
scores = bm25Model.get_scores(text)

In [58]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    scores = bm25Model.get_scores(text)
    top_query = np.argsort(scores)[::-1][:10]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        text += (sentenece_arr[query])
        
    scores = bm25Model.get_scores(text)
    total_scores.append(scores)

search_result = np.ones((20,300))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:300]
    search_result[cnt] += keys
search_result = search_result.astype(np.int)

0 通姦在刑法上應該除罪化 ['通姦', '在', '刑法', '上', '應該', '除罪', '化']
1 應該取消機車強制二段式左轉(待轉) ['應該', '取消', '機車', '強制', '二段式', '左轉', '(', '待轉', ')']
2 支持博弈特區在台灣合法化 ['支持', '博弈', '特區', '在', '台灣', '合法化']
3 中華航空空服員罷工是合理的 ['中華', '航空', '空服員', '罷工', '是', '合理', '的']
4 性交易應該合法化 ['性交易', '應該', '合法化']
5 ECFA早收清單可（有）達到其預期成效 ['ECFA', '早收', '清單', '可', '（', '有', '）', '達到', '其', '預期', '成效']
6 應該減免證所稅 ['應該', '減', '免證', '所稅']
7 贊成中油在觀塘興建第三天然氣接收站 ['贊成', '中油', '在', '觀塘', '興建', '第三', '天然氣', '接收站']
8 支持中國學生納入健保 ['支持', '中國', '學生', '納入', '健保']
9 支持臺灣中小學（含高職、專科）服儀規定（含髮、襪、鞋）給予學生自主 ['支持', '臺灣', '中小學', '（', '含', '高職', '、', '專科', '）', '服儀', '規定', '（', '含', '髮', '、', '襪', '、', '鞋', '）', '給予', '學生', '自主']
10 不支持使用加密貨幣 ['不', '支持', '使用', '加密', '貨幣']
11 不支持學雜費調漲 ['不', '支持', '學雜費', '調漲']
12 同意政府舉債發展前瞻建設計畫 ['同意', '政府', '舉債', '發展', '前瞻', '建', '設計', '畫']
13 支持電競列入體育競技 ['支持', '電競', '列入', '體育競技']
14 反對台鐵東移徵收案 ['反對', '台鐵', '東移', '徵收', '案']
15 支持陳前總統保外就醫 ['支持', '陳', '前', '總統', '保外就醫']
16 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', 

## tfidf model

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(sentenece_arr)
new_corpus = [ dictionary.doc2bow(text) for text in sentenece_arr]

In [ ]:
from gensim.models import TfidfModel
tfidf = TfidfModel(new_corpus)

In [ ]:
doc_dict = {}
for idx, text in enumerate(new_corpus):
    all_words = ([i[0] for i in text])
    
    for word in all_words:
        if word in doc_dict.keys():
            doc_dict[word].append(idx)
        else:
            doc_dict[word]= [idx]

In [14]:
total_scores = list()

for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    print(test_id ,text_q, text)
    
    text_bow = dictionary.doc2bow(text) 
    keys, values = zip(*tfidf[text_bow])
    
    for cnt, key in enumerate(keys):
        if cnt == 0:
            queried_doc = set(doc_dict[key])
        else:
            queried_doc = set(doc_dict[key]) | queried_doc
    
    cosine_sim_score = list()
    for cnt, doc in enumerate(queried_doc):
        tfidf_score = np.zeros((len(keys)))
        result = sentenece_arr[doc]
        result = dictionary.doc2bow(result)
        result_dict = dict(tfidf[result])
        
        for idx, i in enumerate(keys):
            if i in result_dict.keys():
                tfidf_score[idx] = result_dict[i]
        
        tfidf_score, values  = np.array(tfidf_score).reshape(1,-1), np.array(values).reshape(1,-1)
        score = cosine_similarity(tfidf_score, values)
        cosine_sim_score.append((doc, score[0][0]))
        
    total_scores.append(cosine_sim_score)

0 通姦在刑法上應該除罪化 ['通姦', '在', '刑法', '上', '應該', '除罪', '化']
1 應該取消機車強制二段式左轉(待轉) ['應該', '取消', '機車', '強制', '二段式', '左轉', '(', '待轉', ')']
2 支持博弈特區在台灣合法化 ['支持', '博弈', '特區', '在', '台灣', '合法化']
3 中華航空空服員罷工是合理的 ['中華', '航空', '空服員', '罷工', '是', '合理', '的']
4 性交易應該合法化 ['性交易', '應該', '合法化']
5 ECFA早收清單可（有）達到其預期成效 ['ECFA', '早收', '清單', '可', '（', '有', '）', '達到', '其', '預期', '成效']
6 應該減免證所稅 ['應該', '減', '免證', '所稅']
7 贊成中油在觀塘興建第三天然氣接收站 ['贊成', '中油', '在', '觀塘', '興建', '第三', '天然氣', '接收站']
8 支持中國學生納入健保 ['支持', '中國', '學生', '納入', '健保']
9 支持臺灣中小學（含高職、專科）服儀規定（含髮、襪、鞋）給予學生自主 ['支持', '臺灣', '中小學', '（', '含', '高職', '、', '專科', '）', '服儀', '規定', '（', '含', '髮', '、', '襪', '、', '鞋', '）', '給予', '學生', '自主']
10 不支持使用加密貨幣 ['不', '支持', '使用', '加密', '貨幣']
11 不支持學雜費調漲 ['不', '支持', '學雜費', '調漲']
12 同意政府舉債發展前瞻建設計畫 ['同意', '政府', '舉債', '發展', '前瞻', '建', '設計', '畫']
13 支持電競列入體育競技 ['支持', '電競', '列入', '體育競技']
14 反對台鐵東移徵收案 ['反對', '台鐵', '東移', '徵收', '案']
15 支持陳前總統保外就醫 ['支持', '陳', '前', '總統', '保外就醫']
16 年金改革應取消或應調降軍公教月退之優存利率十八趴 ['年金', '改革', '應', '取消', '或應', '調降', 

In [48]:
search_result = np.ones((20,300))
for cnt,i in enumerate(total_scores):
    keys , values = zip(*sorted(i, key= lambda x: -x[1])[:300])
    search_result[cnt] += keys
search_result = search_result.astype(np.int)

In [65]:
df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(300):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i]

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))
    
fname = 'simple9.csv'
df.to_csv('output/' + fname, index=False)

In [69]:
idx=search_result[1][0]
content_list[idx-1]

'在台灣騎機車常常需要兩段式左轉，但部分騎士認為，這樣子其實是在削減機車族的完整路權，要求交通部要廢除兩段式左轉。對此駕訓班教練認為，兩段式左轉比較安全應該存在；學者則認為，兩段式左轉其實有保留價值，政府應該不要強制兩段式左轉，讓騎士自由選擇要直接左轉或是兩段式左轉，就可以讓不敢直接左轉的騎士，作為轉彎的緩衝。\n駕訓班練習場，學員正在練習騎機車，教練不斷叮嚀，必須二段式左轉的路段，一定要依照規定行駛。\n駕訓班教練吳泰利：「我們標線有禁行機車的，都要兩段式左轉，白牌都要這樣子做。」\n教練提出，沒有按照兩段式左轉容易發生危險，實際案例其實不斷出現，今年8月台北市南港發生機車對撞車禍，就是疑似機車騎士搶快，直接左轉撞上對向重機車，造成腿部骨折。但兩段式左轉要順利停等，又不影響後方來車，其實注意事項還不少。\n駕訓班教練簡美枝：「你打了右邊方向燈擺頭，那我行進到這裡的地方，我還要確認兩方，看一下速度放慢，到這地方到格子。」\n教練戴上頭帶式攝影機親自示範，遇到必須兩段式左轉的路口，要打右轉方向燈，左右擺頭，然後再轉進待轉區，只要基本動作做確實，其實並不麻煩。\n駕訓班教練吳泰利：「因為路就是那麼大，你小車一定會鑽，大車也不會讓，我們的用車習慣就是這樣子，你假如有規劃的話，他就乖乖地在那邊待轉。」\n不過對於國內部分的機車騎士來說，認為兩段式左轉影響機車騎士的路權完整性。\n機車路權促進會理事長陳宏益：「為了禁行機車的配套，因為禁行機車以後，內側車道被禁行以後，他就沒辦法左轉，所以就拿個兩段式左轉，來搭配著使用，反正一個錯誤，加上另一個錯誤來配套來處理。」\n機車族強調，有些機車待轉區太過狹小，就容易造成交通堵塞，有時候要準備待轉，卻被後方車輛以為要右轉，反而提高車禍機率。\n機車路權促進會成員林詠軒：「因為整個車道裡面都是要右轉的時候，你待在車道的右側，左邊的車子他以為你也是要右轉，這時候會有方向衝突的問題。」\n機車路權促進會成員吳祥瑀：「我曾經遇過因為我要直走，結果有一台兩段式左轉的機車，他就在我的左邊，那他可能以為我要右轉，可是我騎在車道正中間，結果他就突然從我左邊往右切過來。」\n也因此這群機車騎士聚集起來，不斷呼籲交通部應該取消機車兩段式左轉，同時取消禁行機車的路段，騎士們認為這樣機車可以恢復完整路權，同時也讓機車行駛不再綁手綁腳。\n機車路權促進會副理事長